In [ ]:
!pip install backtesting 
!pip install yfinance
!pip install ta-lib-binary 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173819 sha256=f0bf6c81b3050565ad289f79ce8198332cdbd5f80aab1151117e6526f6628bd9
  Stored in directory: /root/.cache/pip/wheels/cd/5c/f7/aafe95c37f8b07f838fb0a8cb3177de23a38c09cbd10b447b8
Successfully built backtesting
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 13

In [85]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import yfinance as yf 
import talib  
import datetime 
import pandas as pd

In [158]:
# ticker and the start and end dates for testing
ticker =  input("Enter ticker: ")
start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2023, 1, 1)
df=yf.download( ticker , start=start,end=end)
df

Enter ticker: ^GDAXI
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,7689.459961,7789.939941,7689.459961,7778.779785,7778.779785,140345500
2013-01-03,7762.129883,7777.020020,7737.979980,7756.439941,7756.439941,86174100
2013-01-04,7740.990234,7779.669922,7727.549805,7776.370117,7776.370117,101577000
2013-01-07,7760.009766,7767.549805,7713.419922,7732.660156,7732.660156,151695400
2013-01-08,7710.700195,7754.379883,7683.479980,7695.830078,7695.830078,130425500
...,...,...,...,...,...,...
2022-12-23,13945.589844,14000.679688,13874.500000,13940.929688,13940.929688,28738700
2022-12-27,14047.419922,14063.139648,13966.349609,13995.099609,13995.099609,22975000
2022-12-28,14013.719727,14018.469727,13914.620117,13925.599609,13925.599609,27583800


In [159]:
def EMA_Backtesting(values, n):
    """
    Return exponential moving average of `values`, at
    each step taking into account `n` previous values.
    """
    close = pd.Series(values)
    return talib.EMA(close, timeperiod=n) 

In [160]:
class EmaCrossStrategy(Strategy):
    
    # Define the two EMA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 10
    
    def init(self):
        # Precompute two moving averages
        self.ema1 = self.I(EMA_Backtesting, self.data.Close, self.n1)
        self.ema2 = self.I(EMA_Backtesting, self.data.Close, self.n2)
    
    def next(self):       
        # If ema1 crosses above ema2, buy the asset
        if crossover(self.ema1, self.ema2):
            self.position.close()
            self.buy()

        # Else, if ema1 crosses below ema2, sell it
        elif crossover(self.ema2, self.ema1):
            self.position.close()
            self.sell()


In [161]:
bt = Backtest(df, EmaCrossStrategy, cash=100_000, commission=.001)
stats = bt.run()

In [162]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Return [%]',
                    constraint=lambda param: param.n1 < param.n2)
stats

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 223 ms, sys: 41.7 ms, total: 264 ms
Wall time: 4.95 s


Start                     2013-01-02 00:00:00
End                       2022-12-30 00:00:00
Duration                   3649 days 00:00:00
Exposure Time [%]                   98.973144
Equity Final [$]                 92291.541375
Equity Peak [$]                 107480.021308
Return [%]                          -7.708459
Buy & Hold Return [%]               78.994524
Return (Ann.) [%]                   -0.795197
Volatility (Ann.) [%]               16.859281
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -34.77105
Avg. Drawdown [%]                   -12.75622
Max. Drawdown Duration     2514 days 00:00:00
Avg. Drawdown Duration      722 days 00:00:00
# Trades                                  107
Win Rate [%]                         32.71028
Best Trade [%]                      19.989272
Worst Trade [%]                     -6.554218
Avg. Trade [%]                    

In [163]:
stats._strategy 

<Strategy EmaCrossStrategy(n1=10,n2=20)>

In [164]:
bt.plot()

Row(id='12173', ...)

In [165]:
stats['_trades']  # Contains individual trade data 

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-13,26,43,7573.788747,7654.669922,-1051.455272,-0.010679,2013-02-07,2013-03-04,25 days
1,12,43,66,7662.324592,7669.040039,80.585367,0.000876,2013-03-04,2013-04-08,35 days
2,-12,66,83,7661.370999,7905.350098,-2927.749184,-0.031845,2013-04-08,2013-05-02,24 days
3,12,83,113,7913.255448,8023.370117,1321.376033,0.013915,2013-05-02,2013-06-13,42 days
4,-12,113,135,8015.346747,8237.919922,-2670.878098,-0.027768,2013-06-13,2013-07-15,32 days
...,...,...,...,...,...,...,...,...,...,...
102,-5,2389,2418,13543.882989,13183.240234,1803.213774,0.026628,2022-06-14,2022-07-25,41 days
103,6,2418,2441,13196.423475,13328.490234,792.400559,0.010008,2022-07-25,2022-08-25,31 days
104,-6,2441,2481,13315.161744,12685.160156,3780.009527,0.047315,2022-08-25,2022-10-20,56 days
105,6,2481,2524,12697.845316,13829.900391,6792.330445,0.089153,2022-10-20,2022-12-20,61 days
